# Exploratory data analysis of Los Angeles Police Department crime data

## Importing libraries

In [ ]:
import pandas as pd
import plotly.express as px 

#### Configuring pandas options

In [ ]:
pd.options.display.float_format = '{:.6f}'.format #https://medium.com/@anala007/float-display-in-pandas-no-more-scientific-notation-80e3dd28eabe
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 500)

#### Loading Csv files

In [ ]:
df1 = pd.read_csv('Crime_Data_from_2010_to_2019_20240229.csv')
df2 = pd.read_csv('Crime_Data_from_2020_to_Present_20240229.csv')

### Merging both datasets

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
df = pd.concat([df1, df2])
df.shape

#### Now checking columns in both datasets

In [ ]:
df.columns

In [ ]:
df1.columns

#### Renaming `AREA ` to `AREA`

In [ ]:
df1.rename(columns={ "AREA " :"AREA"}, inplace=True)
df1.columns

#### Concating both datasets

In [ ]:
df = pd.concat([df1, df2])
df.shape

In [ ]:
df.head()

#### Dropping duplicate rows

In [ ]:
df.drop_duplicates(keep=False,inplace=True)

### Finding rows containing null values

In [ ]:
df.isnull().sum()

Converting values to percentage values.

In [ ]:
(df.isnull().sum()).sort_values(ascending=False)/df.shape[0] * 100

#### Dropping rows that are having null values in one of these columns : `Mocodes`, `Status`, `Crm Cd 1`, `Premis Cd`

In [ ]:
df.dropna(subset=['Premis Cd', 'Mocodes', 'Status', 'Crm Cd 1', 'Premis Cd'], inplace=True)

Let's check null values again

In [ ]:
(df.isnull().sum()).sort_values(ascending=False)/df.shape[0] * 100

### Now filling null values for remaining columns 

In [ ]:
values = { 
    "Crm Cd 4" : "N/A",
    "Crm Cd 3" : "N/A",
    "Crm Cd 2" : "N/A",
    "Cross Street" : "N/A",
    "Weapon Desc" : "UNKNOWN",
    "Weapon Used Cd" : 0,
    "Premis Desc" : "N/A",
    "Vict Descent":"X",
    "Vict Sex" : "X"
}
df.fillna(value=values, inplace=True)
(df.isnull().sum()).sort_values(ascending=False)/df.shape[0] * 100

Let's check unique values of victim's gender

In [ ]:
df["Vict Sex"].unique()

It's better to replace `-` with `X`

In [ ]:
df['Vict Sex'].replace(['-'], "X", inplace=True)
df["Vict Sex"].unique()

Let's check vicitim descent

In [ ]:
df['Vict Descent'].unique()

Replacing `-` with word `Unknwon`

In [ ]:
df['Vict Descent'].replace('-','Unknown', inplace=True)
df['Vict Descent'].unique()

Now converting codes to meaningful names

In [ ]:
def populate_vict_desc(df):
    mapping={ 
        "A": "Other Asian",
        "B" : "Black",
        "C":"Chinese",
        "D" : "Cambodian",
        "F" : "Filipino",
        "G" : "Guamanian",
        "H" : "Hispanic/Latin/Mexican",
        "I" : "American Indian/Alaskan Native",
        "J" : "Japanese",
        "K" : "Korean",
        "L" : "Laotian",
        "O" : "Other",
        "P" : "Pacific Islander",
        "S" : "Samoan",
        "U" : "Hawaiian",
        "V" : "Vietnamese",
        "W" : "White",
        "X" : "Unknown",
        "Z" : "Asian Indian"
    }
    for i in mapping:
        df.loc[df["Vict Descent"]==i, "Vict Descent"]=mapping[i]
    return df
populate_vict_desc(df).head()

Now dropping columns that are not required for EDA.

In [ ]:
df.drop(columns=['DR_NO','Part 1-2', 'Mocodes','Status', 'Crm Cd 1','Crm Cd 2','Crm Cd 3', 'Crm Cd 4', 'Cross Street'],inplace=True)

Generating new columns that can be useful for future EDA

In [ ]:
df["DATETIME OCC"] = pd.to_datetime(df["DATE OCC"], format="%m/%d/%Y %I:%M:%S %p")
df["DATETIME Rptd"] = pd.to_datetime(df["Date Rptd"], format="%m/%d/%Y %I:%M:%S %p")
df["DAY OCC"] = df["DATETIME OCC"].dt.day_name()
df["MONTH OCC"] = df["DATETIME OCC"].dt.month_name()
df["YEAR OCC"] = df["DATETIME OCC"].dt.year
df['TIME OCC'] = df['TIME OCC'].astype(str).str.zfill(4)
df['HOUR OCC'] = df['TIME OCC'].apply(lambda t: int(t[:2]))
df.head()


Taking a copy of df

In [ ]:
df_copy = df # Taking copy of clean dataframe

Let's check dataframe's information

In [ ]:
df.info()

Let's check data statistics

In [ ]:
df.describe()

Dropping values with negative age values

In [ ]:
df.drop(df[df["Vict Age"] < 0].index, inplace=True)

In [ ]:
df.describe()

### Victim information analysis

Let's analyze cases by victim's gender, age and descent.

In [ ]:
victims_by_gender = df.groupby(["Vict Sex"]).size().reset_index(name="Incidents")
fig=px.pie(victims_by_gender, values='Incidents', names="Vict Sex", title="Incidents by victims gender")
fig.update_traces(textinfo='percent+label')
fig.show()

Now let's visulize victims's age and gender

In [ ]:
age_group_data = df.copy()
age_group_data=age_group_data.groupby(['Vict Age', 'Vict Sex']).size().reset_index(name='Incidents')
age_color_scheme = {"X" : "black", "M" : "blue", "F" : "pink", "H" : "red", "N" : "green"}
px.histogram(age_group_data, x="Vict Age", y="Incidents", color="Vict Sex", text_auto=True, color_discrete_map=age_color_scheme)

Let's visualize victim's descent

In [ ]:
fig1=px.histogram(df, x='Vict Descent')
fig1.update_layout(yaxis_title="Incidents")
fig1.update_traces( hovertemplate=None)
fig1.add_pie()
vict_descent_data=df.groupby(['Vict Descent']).size().sort_values(ascending=False).reset_index(name="Incidents")
fig2 = px.pie(vict_descent_data, values="Incidents", names="Vict Descent", title="Victim Descent By %")
fig1.show()
fig2.show()

Now let's check victim descent wise age.

In [ ]:
for i in df['Vict Descent'].unique():
    victim_data = df[df['Vict Descent']==i].groupby(["Vict Age", "Vict Sex"]).size().reset_index(name="Incidents")
    victim_data['Incidents Percentage']=(victim_data['Incidents'] / victim_data['Incidents'].sum()) * 100
    fig = px.histogram(victim_data, x="Vict Age", y="Incidents Percentage", color="Vict Sex", barmode="group", title=i, color_discrete_map=age_color_scheme, hover_data=["Incidents"])
    fig.show()

### Crime Area Information

Visulizing crime areas with vict descent.

In [ ]:
px.histogram(df, x="AREA NAME", color="Vict Descent",text_auto=True).update_xaxes(categoryorder="total descending") #https://community.plotly.com/t/plotly-express-histogram-any-way-to-sort-bar-by-value/23905/4

From above graph we can observere that most crimes occurred in `77th street` and `Hollenbeck` had lowest crime rates.

Also `Hispanic/Latin/Mexican`, `White` and `Black` people got affected most by crimes in almost all areas.

#### Now visualizing areas by victims gender

In [ ]:
px.histogram(df, x="AREA NAME", color="Vict Sex", color_discrete_map=age_color_scheme, text_auto=True).update_xaxes(categoryorder="total descending") #https://community.plotly.com/t/plotly-express-histogram-any-way-to-sort-bar-by-value/23905/4

From above data we can observe that in most areas majority of victims are male, only for `77th street`, `Southwest` and `Southeast`number female victims are greater than males. 

#### Now let's take a look at premis information

In [ ]:
premis_wise_crimes = df.groupby(["Premis Desc"]).size().reset_index(name="Cases")
fig=px.pie(premis_wise_crimes, values="Cases", names="Premis Desc")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

#### Visualizing premis information area wise

In [ ]:
area_wise_data = df.groupby(['AREA NAME', 'Premis Desc']).size().reset_index(name="Cases")
for area in area_wise_data["AREA NAME"].unique():
    fig = px.bar(area_wise_data[area_wise_data["AREA NAME"] == area], x="Premis Desc", y="Cases", text_auto=True).update_xaxes(title="Type of Crimes",categoryorder="total descending", range=(0,10)).update_yaxes(title="Cases")
    fig.update_layout(title="Crimes in " + area)
    fig.show()
# px.histogram(area_wise_data, x="AREA NAME", color="Premis Desc", text_auto=True).update_xaxes(categoryorder="total descending") #https://community.plotly.com/t/plotly-express-histogram-any-way-to-sort-bar-by-value/23905/4

### Visualizing type of crime

In [ ]:
cases_by_type = df.groupby(["Crm Cd Desc", "Vict Sex", "Vict Descent", "AREA NAME"]).size().reset_index(name="Cases")
fig=px.pie(cases_by_type, values="Cases", names="Crm Cd Desc", height=800)
fig.update_traces(textposition='inside', textinfo='percent+label+value')

In [ ]:
for area in df["AREA NAME"].unique():
    fig = px.histogram(df[df["AREA NAME"]==area], x="Crm Cd Desc", title="Crimes in %s"%area, text_auto=True).update_xaxes(title="Crimes",categoryorder="total descending", range=(0,10)).update_yaxes(title="Cases")
    fig.show()

Let's check crimes based on premis and crime type

In [ ]:
premis_wise_crimes = df.groupby(["Premis Desc", "Crm Cd Desc"]).size().reset_index(name="Cases")
premis_wise_crimes = premis_wise_crimes[premis_wise_crimes["Cases"] > 10000] # Taking only rows which crime incidents are greater than 10000
# px.histogram(df, X="Premise Desc")
premis_wise_crimes_pivot = premis_wise_crimes.pivot(index="Premis Desc", columns="Crm Cd Desc", values="Cases")
premis_wise_crimes_pivot.head()

Replacing null values with 0

In [ ]:
premis_wise_crimes_pivot.fillna(value=0, inplace=True)
premis_wise_crimes_pivot.head()

Plotting heatmap to visualize the data

In [ ]:
fig = px.imshow(premis_wise_crimes_pivot,
                labels=dict(x="Type of crime", y="Premis description", color="Number of Crimes"),
                x=premis_wise_crimes_pivot.columns,
                y=premis_wise_crimes_pivot.index
                )

fig.update_xaxes(side="top")
fig.update_layout(title="Premise wise Crime Types",
                  yaxis=dict(title="Type of crime"),
                  xaxis=dict(title="Premis description"),
                  height=800)
fig.show()

In [ ]:

crime_by_area_and_day = df.groupby(['DAY OCC', "AREA NAME"]).size().reset_index(name="Cases")
crime_pivot = crime_by_area_and_day.pivot(index='AREA NAME', columns='DAY OCC', values='Cases')

fig = px.imshow(crime_pivot,
                labels=dict(x="Hour of the Day", y="Day of the Week", color="Number of Crimes"),
                x=crime_pivot.columns,
                y=crime_pivot.index,
                color_continuous_scale="viridis")

fig.update_xaxes(side="top")
fig.update_layout(title="Crime Occurrences by Day and Time",
                  xaxis=dict(title="Hour of the Day"),
                  yaxis=dict(title="Day of the Week"),
                  height=800)

#### Let's check time of crimes

In [ ]:
year_wise_crime = df["YEAR OCC"].value_counts().reset_index(name="Cases").sort_values("YEAR OCC")
px.line(year_wise_crime, x="YEAR OCC", y="Cases",  markers=True)

In [ ]:
year_wise_crime = df.groupby(["YEAR OCC", "AREA NAME"]).size().reset_index(name="Cases").sort_values("YEAR OCC")
px.line(year_wise_crime, x="YEAR OCC", y="Cases",  markers=True, color="AREA NAME")

##### Now check which days had more crime incidents

In [ ]:
def sort_by_dayname(df, col="DAY OCC") : 
    day_name_to_number = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}
    df[col + "NUMBER"] = df[col].apply(lambda x : day_name_to_number[x])
    df.sort_values(by=col+"NUMBER", inplace=True)
    df.drop(columns=[col+'NUMBER'], inplace=True)
    return df

In [ ]:

crime_by_day = df.groupby(['DAY OCC']).size().reset_index(name="Cases")
px.line(sort_by_dayname(crime_by_day), x="DAY OCC", y="Cases", markers=True)

In [ ]:
crime_by_day = df.groupby(['DAY OCC', "HOUR OCC"]).size().reset_index(name="Cases")
# Pivot the DataFrame for visualization
crime_pivot = crime_by_day.pivot(index='DAY OCC', columns='HOUR OCC', values='Cases')

# Create heatmap using Plotly
fig = px.imshow(crime_pivot,
                labels=dict(x="Hour of the Day", y="Day of the Week", color="Number of Crimes"),
                x=crime_pivot.columns,
                y=crime_pivot.index,
                color_continuous_scale="viridis")

# Customize layout
fig.update_layout(title="Crime Occurrences by Day and Time",
                  xaxis=dict(title="Hour of the Day"),
                  yaxis=dict(title="Day of the Week"))

Let's check month wise crimes

In [ ]:
year_month_wise_crime = df.groupby(["YEAR OCC", "MONTH OCC"]).size().reset_index(name="Cases").sort_values("YEAR OCC")
px.histogram(year_month_wise_crime, x="MONTH OCC", y="Cases", text_auto=True).update_xaxes(title="Month",categoryorder="total descending").update_yaxes(title="Cases")

In [ ]:
year_month_wise_crime.describe()

In [ ]:
for year in year_month_wise_crime["YEAR OCC"].unique() : 
    fig = px.histogram(year_month_wise_crime[year_month_wise_crime["YEAR OCC"]==year], x="MONTH OCC", y="Cases", text_auto=True).update_xaxes(title="Month", categoryorder="total descending").update_yaxes(title="Crimes")
    fig.update_layout(title="Crimes in " + str(year))
    fig.show()

### Now checking status of these crimes

In [ ]:
fig=px.pie(df, names="Status Desc", height=800)
fig.update_traces(textinfo='percent+label+value')
fig.show()

### Now checking weapon used in crimes

In [ ]:
fig=px.pie(df, names="Weapon Desc", height=800)
fig.update_traces(textposition='inside', textinfo='percent+label+value')
fig.show()

Let's try removing unknown weapon

In [ ]:
fig=px.pie(df[df["Weapon Desc"]!= "UNKNOWN"], names="Weapon Desc", height=800)
fig.update_traces(textposition='inside', textinfo='percent+label+value')
fig.show()

### Let's visualize crimetype with weapon

In [ ]:
crimes_weapons_data = df.groupby(["Weapon Desc", "Crm Cd Desc"]).size().reset_index(name="Cases")
crimes_weapons_data = crimes_weapons_data[crimes_weapons_data["Cases"] > 5000] # Taking only rows which crime incidents are greater than 10000
# # px.histogram(df, X="Premise Desc")
crimes_weapons_data_pivot = crimes_weapons_data.pivot(index="Weapon Desc", columns="Crm Cd Desc", values="Cases")
crimes_weapons_data_pivot.fillna(value=0, inplace=True)
crimes_weapons_data_pivot.head()

In [ ]:
fig = px.imshow(crimes_weapons_data_pivot,
                labels=dict(x="Weapon Used", y="Type of crime", color="Number of Crimes"),
                x=crimes_weapons_data_pivot.columns,
                y=crimes_weapons_data_pivot.index
                )

fig.update_xaxes(side="top")
fig.update_layout(title= "Crime Types vs Weapon Used",
                  yaxis=dict(title="Type of crime"),
                  xaxis=dict(title="Premis description"),
                  height=800)
fig.show()